#  Data Science Learning Journey  
*Curiosity to Capability — One Notebook at a Time*

---
Compiled and authored by **Partho Sarothi Das**   
	Dhaka, Bangladesh  
	Bachelor's & Master's in Statistics  
	Investment Banking Professional → Aspiring Data Scientist 
    
---

# Example of Window Functions

### RANK

```sql
SELECT *,
RANK() OVER(PARTITION BY branch ORDER BY marks DESC)
FROM partho.marks
```

### DENSE_RANK

```sql
SELECT *,
RANK() OVER(PARTITION BY branch ORDER BY marks DESC)
DENSE_RANK() OVER(PARTITION BY branch ORDER BY marks DESC) AS 'dense_rank'
FROM partho.marks
```
> **Note:** Check ECE branch

### ROW_NUMBER

```sql
SELECT *,
ROW_NUMBER() OVER(PARTITION BY branch)
FROM partho.marks
```

### CONCAT

```sql
SELECT *,
CONCAT(branch, '-', ROW_NUMBER() OVER(PARTITION BY branch)) AS "Dept_roll"
FROM partho.marks
```

### FIRST_VALUE

```sql
SELECT *,
FIRST_VALUE(marks) OVER(PARTITION BY branch ORDER BY marks DESC)
FROM partho.marks
```

### LAST_VALUE

```sql
SELECT *,
LAST_VALUE(marks) OVER(PARTITION BY branch
						ORDER BY marks desc
						ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING)
FROM partho.marks
```

### NTH_VALUE

```sql
SELECT *,
NTH_VALUE(marks, 2) OVER(PARTITION BY branch
						ORDER BY marks desc
						ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING)
FROM partho.marks
```

### LAG

```sql
SELECT *,
LAG(marks) OVER(ORDER BY student_id)
FROM partho.marks
```

### LEAD

```sql
SELECT *,
LAG(marks) OVER(ORDER BY student_id) AS 'by_lag',
LEAD(marks) OVER(ORDER BY student_id) AS 'by_lead'
FROM partho.marks
```

### *Cumulative Average*

```sql
SELECT 
ROW_NUMBER() OVER(ORDER BY id) AS match_no,
SUM(batsman_run) AS runs,
SUM(SUM(batsman_run)) OVER (ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS career_runs,
AVG(SUM(batsman_run)) OVER (ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS career_avg
FROM partho.ipl
WHERE batter = "V Kohli"
GROUP BY ID
```

### *Moving Average (Runing Average)*

```sql
SELECT
ROW_NUMBER() OVER(ORDER BY id) AS match_no,
SUM(batsman_run) AS runs,
AVG(SUM(batsman_run)) OVER (ROWS BETWEEN 9 PRECEDING AND CURRENT ROW) AS moving_avg
FROM partho.ipl
WHERE batter = "V Kohli"
GROUP BY ID
```

### Percent_rank

*Percent rank is rank by 0 to 1 of all the values*

```sql
SELECT marks,
PERCENT_RANK () OVER(ORDER BY marks) AS pr
FROM partho.marks
```

**Example:** Find Q1, Q2 and Q3 of partho.marks dataset.
```sql
WITH ranked_marks AS(
					SELECT marks,
					PERCENT_RANK () OVER (ORDER BY marks) AS pr
					FROM partho.marks)
SELECT 
(SELECT MARKS FROM ranked_marks WHERE pr >= 0.25 ORDER BY pr LIMIT 1) AS Q1,
(SELECT marks FROM ranked_marks WHERE pr >= 0.50 ORDER BY pr LIMIT 1) AS Q2,
(SELECT marks FROM ranked_marks WHERE pr >= 0.75 ORDER BY pr LIMIT 1) AS Q3
```

### Segmentation

```sql
SELECT *, 
NTILE(3) OVER(ORDER BY marks DESC) AS segmentation 
FROM partho.marks;
```

**Example:** Segment into three group as budget, mid-range, premium of the smartphone according to price.

```sql
SELECT brand_name, price,
CASE
	WHEN bucket = 1 THEN 'budget'
    WHEN bucket = 2 THEN 'mid-range'
    WHEN bucket = 3 THEN 'premium'
END AS phone_type
FROM (
			SELECT brand_name, price,
			NTILE(3) OVER(ORDER BY price) AS bucket 
			FROM partho.smartphones) t
```

### Cumulative Distribution Function

**Example:**

```sql
SELECT *,
CUME_DIST() OVER(ORDER BY marks) AS 'Percentile_Score'
FROM partho.marks;
```

**Example:** Find all students from the partho.marks table whose marks are in the top 10% based on cumulative distribution.

```sql
SELECT * FROM(
		SELECT *,
		CUME_DIST() OVER(ORDER BY marks) AS 'Percentile_Score'
		FROM partho.marks) T
WHERE Percentile_Score> 0.9
```

### Partition by Multiple Columns

**Example:**

### Median

**Example: 1** Find the median marks of all the students.

```sql

```

**Example: 2** Find branchwise median marks of all the students.

```sql

```

# Excersice

*1. Find all the students who have marks higher than the avg marks of their respective branch.*

```sql
SELECT * FROM (SELECT *,
AVG(marks) OVER (PARTITION BY branch) AS 'branch_avg'
FROM partho.marks) T
WHERE T.marks > T.branch_avg
```

*2. Find top 2 most paying customersd of each month.*

```sql
SELECT * FROM (SELECT  MONTHNAME(date) AS 'month', user_id, SUM(amount) AS 'total',
				RANK() OVER (PARTITION BY MONTHNAME(date) ORDER BY SUM(amount) DESC) AS 'month_rank' 
				FROM zomato.orders
				GROUP BY MONTHNAME(date), user_id
				ORDER BY MONTHNAME(date)) t
				WHERE t.month_rank<3
                ORDER BY month DESC
```

*3. Create roll no from branch and marks.*

```sql

```

*1. Find the branch toppers*

```sql
SELECT name, branch, marks
FROM (SELECT *,
	  FIRST_VALUE(marks) OVER (PARTITION BY branch ORDER BY marks DESC) AS topper_marks
	  FROM partho.marks
      ) AS t
WHERE marks = topper_marks;
```

*2. FRAME Clause*

*3. Find the last guy of each branch*

```sql
SELECT * 
FROM(
	SELECT *,
	LAST_VALUE(marks) OVER(PARTITION BY branch ORDER BY marks DESC
	ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) 
	AS lowest_marks
	FROM partho.marks
	) AS t
WHERE marks = lowest_marks
```

*4. Alternate way of writing window functions*

*5. Find he 2nd last guy of each branch*

```sql
SELECT name, branch, value_2nd
FROM (
	SELECT *, 
	NTH_VALUE(MARKS, 2) OVER(PARTITION BY branch ORDER BY marks DESC
						ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
						) AS value_2nd
	FROM partho.marks) AS t
WHERE marks = value_2nd
```

**Example:** *Find the month on month revenue growth.*

```sql
WITH monthly_orders AS (SELECT 
						MONTH(date) AS month_num,
						MONTHNAME(date) AS month_name,
						SUM(amount) AS total_amount
						FROM zomato.orders
						GROUP BY MONTH(date), MONTHNAME(date)
)

SELECT 
    month_name, total_amount,
    ROUND(
        ((total_amount - LAG(total_amount) OVER (ORDER BY month_num)) 
         / LAG(total_amount) OVER (ORDER BY month_num)) * 100, 2) AS pct_growth
FROM monthly_orders
ORDER BY month_num;
```

### ipl dataset

*1. Who are the top 5 batsman in each ipl team.*

```sql
SELECT * FROM(
		SELECT BattingTeam, batter, SUM(batsman_run) AS total_runs,
		DENSE_RANK() OVER(PARTITION BY BattingTeam ORDER BY SUM(batsman_run) DESC) AS rank_within_team
		FROM partho.ipl
		GROUP BY BattingTeam, batter) t
WHERE t.rank_within_team < 6
```

*2. How many runs V Kohli made upto his 5th, 100th and 150th match (Use cumulative sum)*

```sql
SELECT * FROM (
	SELECT ID, ROW_NUMBER() OVER(ORDER BY ID) AS match_no,
	SUM(batsman_run) AS runs,
	SUM(SUM(batsman_run)) over(ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS career_runs
	FROM partho.ipl
	WHERE batter = 'V Kohli'
	GROUP BY ID
    ) t
WHERE t.match_no = 5 or t.match_no = 100 or t.match_no = 150
```

*Find the percentage of total sell amount each restaurants.*

```sql
SELECT t.f_id, f_name, (total_value/SUM(total_value) OVER()) * 100  AS percent_of_total
FROM(SELECT f_id, SUM(amount) AS total_value
		FROM zomato.orders t1
		JOIN zomato.order_details t2
		ON t1.order_id = t2.order_id
		WHERE r_id = 1
		GROUP BY f_id) t
JOIN zomato.food t3
ON t.f_id = t3.f_id

```
> This is for restaurant id 1, You can change it by change the value of

```sql
> WHERE r_id = 1 
```

### DSE Dataset

*1. Find the monthly trade volume from dse database.*

```sql
SELECT YEAR(Date) AS year, 
MONTHNAME(date) AS month, 
SUM(Volume) AS monthly_volume
FROM dse.dse_market_info
GROUP BY YEAR(date), MONTHNAME(date)
```

*2. Find the monthly trade volume from dse database (Ascending order).*

```sql
SELECT YEAR(Date) AS year,
MONTH(date) AS month, 
MONTHNAME(date) AS month_name, 
SUM(Volume) AS monthly_volume
FROM dse.dse_market_info
GROUP BY YEAR(date), MONTH(date), MONTHNAME(date)
ORDER BY year, month
```

*3. Find out the percentage change of volume monthwise*

```sql
SELECT YEAR(Date) AS year,
MONTH(Date) AS month,
MONTHNAME(Date) AS month_name,
SUM(Volume) AS month_volume,
(SUM(Volume) - LAG(SUM(Volume)) OVER(ORDER BY YEAR(date), MONTH(date))) * 100/SUM(Volume) AS percent_change
FROM dse.dse_market_info
GROUP BY YEAR(date), MONTH(date), MONTHNAME(date)
ORDER BY YEAR, MONTH
```

*4. Find out the weekly percentage change of volume*

```sql
SELECT date, volume,
(Volume - LAG(volume,7) OVER(ORDER BY YEAR(date), MONTH(date), DATE(date))) * 100/Volume AS weekly_percentage_change
FROM dse.dse_market_info
```